In [2]:
synthetic_data_path = './data/synthetic/princeton_tab_synthetic_noDP.csv'

### Downstream Utility Evaluation

In [7]:
import pandas as pd
import json
from transformers import TrainingArguments as HfTrainingArguments
from sdgeval.downstream.classify.train_classifier import TrainingArguments, ModelArguments, Classifier, Arguments
from sdgeval.utils.utils import create_classification_dataset
from sdgeval.utils.filtering import process_df

2025-03-11 15:26:23.249538: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741721183.267785 1530934 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741721183.273328 1530934 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-11 15:26:23.293431: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kramesh3/nltk_data...
[nltk_data]   Package stopwords is a

#### Classification: Creating the dataset

In [8]:
df = pd.read_csv(synthetic_data_path)
df = process_df(df, text_column = 'output_text')
_, _, _ = create_classification_dataset(df, label_column = 'country', output_json_path = './data/benchmark/tab-mapping.json',
                                        output_dir = './data/benchmark/data/princeton_tab_synthetic_country_noDP', multilabel = False, train_ratio = 0.7, test_ratio = 0.15, val_ratio = 0.15)

Data saved to ./data/benchmark/data/princeton_tab_synthetic_country_noDP
Train: 444 samples, Validation: 95 samples, Test: 96 samples


In [ ]:
with open('./data/benchmark/tab-mapping.json') as f:
    data = json.load(f)
    n_labels_task = len(data)
print(n_labels_task)

7


#### Classification: Training the model

In [ ]:
if __name__ == "__main__":
        train_args, model_args = TrainingArguments(), ModelArguments()

        model_args.model_name = 'bert-base-uncased'
        model_args.text_field = 'output_text'
        model_args.label_field = 'Label'
        model_args.path_to_dataset = './data/benchmark/data/princeton_tab_synthetic_country_noDP'
        model_args.path_to_model = './data/benchmark/data/models/princeton_tab_synthetic_country_noDP'
        model_args.n_labels = n_labels_task
        model_args.is_train = True
        model_args.problem_type = 'single_label_classification'
        args = Arguments(train=train_args, model=model_args)

        print("Training:\n")
        obj = Classifier(args = args)
        obj.finetune_model()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training:

Successfully loaded dataset './data/benchmark/princeton_tab_synthetic_country_noDP' from Hugging Face Hub.
Loading base model for fine-tuning...
Preprocessing dataset!


Running tokenizer on dataset:   0%|          | 0/444 [00:00<?, ? examples/s]

/home/kramesh3/.local/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2829: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Running tokenizer on dataset:   0%|          | 0/95 [00:00<?, ? examples/s]

Model training begins...


/home/kramesh3/.local/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.946627,0.390836,0.428571,0.406832,0.852632
2,No log,0.570679,0.522263,0.571429,0.543581,0.936842
3,No log,0.498666,0.522263,0.571429,0.543581,0.936842


/home/kramesh3/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kramesh3/.local/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/kramesh3/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kramesh3/.local/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to

#### Classification: Testing the model

In [ ]:
if __name__ == "__main__":
        train_args, model_args = TrainingArguments(), ModelArguments()
        model_args.is_train = False
        model_args.is_test = True
        model_args.text_field = 'output_text'
        model_args.label_field = 'Label'

        model_args.model_name = 'bert-base-uncased'
        model_args.path_to_model = './data/benchmark/models/princeton_tab_synthetic_country_noDP'
        model_args.path_to_dataset = './data/benchmark/data/princeton_tab_synthetic_country_noDP'
        model_args.path_to_output_csv = './data/benchmark/test-results/princeton_tab_synthetic_noDP_outputs.csv'
        model_args.path_to_aggregated_results = './data/benchmark/model_aggregated_results.csv'

        model_args.n_labels = n_labels_task
        model_args.problem_type = "single_label_classification"
        model_args.retain_columns = ['country', 'input_prompt', 'year']

        args = Arguments(train=train_args, model=model_args)
        print("Testing:\n")
        obj = Classifier(args = args)
        obj.test_model()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Testing:

Successfully loaded dataset './data/benchmark/princeton_tab_synthetic_country_noDP' from Hugging Face Hub.
Checkpoint exists:  ./data/benchmark/tab/models/princeton_tab_synthetic_country_noDP 
Loading model from the checkpoint...
Preprocessing dataset...


Running tokenizer on dataset:   0%|          | 0/96 [00:00<?, ? examples/s]

/home/kramesh3/.local/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2829: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Model evaluation begins...


/home/kramesh3/.local/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Saving file!
Evaluation results:  {'eval_loss': 0.3631312847137451, 'eval_precision': 0.6166666666666667, 'eval_recall': 0.6666666666666666, 'eval_f1': 0.6372549019607843, 'eval_accuracy': 0.96875, 'eval_runtime': 2.3584, 'eval_samples_per_second': 40.705, 'eval_steps_per_second': 0.848}


/home/kramesh3/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#### Classification: Visualizing the results

#### Classification: Fairness audit of the classifier

In [11]:
import pandas as pd
from sdgeval.fairness.metrics import analyze_group_fairness_performance

path_to_outputs = './data/benchmark/test-results/princeton_tab_synthetic_noDP_outputs.csv'
df = pd.read_csv(path_to_outputs)
p_df, f_df = analyze_group_fairness_performance(df, problem_type = "single_label", num_classes = n_labels_task, subgroup_type = "country")
print(p_df.head())
print(f_df.head())

   Accuracy Group Type  Num of Samples  f1_macro  f1_micro
0       0.0        DNK               2       0.0       0.0
1       1.0        GBR              40       1.0       1.0
2       0.0        NOR               1       0.0       0.0
3       1.0        POL              15       1.0       1.0
4       1.0        SWE               7       1.0       1.0
  Group Type  F1-micro-diff  F1-macro-diff
0        DNK            1.0            1.0
1        GBR            1.0            1.0
2        NOR            1.0            1.0
3        POL            1.0            1.0
4        SWE            1.0            1.0


### Descriptive Analysis of Synthetic Data

In [1]:
import nltk
import pandas as pd
from sdgeval.descriptive.descriptor import TextDescriptor
from sdgeval.descriptive.arguments import TextDescriptorArgs
nltk.download('punkt_tab')

/home/kramesh3/.local/lib/python3.9/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(
[nltk_data] Downloading package punkt to /home/kramesh3/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/kramesh3/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/kramesh3/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
from datasets import load_from_disk
synth_df = pd.read_csv(synthetic_data_path)
real_texts = load_from_disk('./data/generate/data/tab')
real_texts = real_texts['train'].select(range(len(synth_df)))
real_texts = real_texts['text']

#### Text length and distributional comparisons

In [3]:
desc_analyze = TextDescriptor(texts = synth_df['output_text'].tolist(), args = TextDescriptorArgs(produce_plot=True), reference_texts = real_texts)

In [4]:
desc_analyze._compare_to_reference_distribution(metrics = ['text-length', 'jaccard', 'cosine'])

Comparing text length...
+-------------------+-----------------------+-----------------------+--------------+
| Metric            |   Text Distribution 1 |   Text Distribution 2 |   Difference |
+===================+=======================+=======================+==============+
| Avg. Length       |               181.416 |              1239.78  |     -1058.36 |
+-------------------+-----------------------+-----------------------+--------------+
| Min Length        |               132     |               389     |      -257    |
+-------------------+-----------------------+-----------------------+--------------+
| Max Length        |               208     |              5144     |     -4936    |
+-------------------+-----------------------+-----------------------+--------------+
| Avg. Unique Words |               114.167 |               459.187 |      -345.02 |
+-------------------+-----------------------+-----------------------+--------------+
Comparing distributions...
Jaccard simil

### Privacy Leakage Assessment

#### Privacy: Defining the entities

In [10]:
import re
import pandas as pd
from datasets import load_from_disk

real_texts = load_from_disk('./data/generate/data/tab')
real_texts = real_texts['train']
synth_df = pd.read_csv(synthetic_data_path)

In [15]:
entities = []
for i in real_texts['annotations']:
    try:
        for annotator in i:
            for entity in i[annotator]['entity_mentions']:
                if(entity['entity_type'] in ['PERSON', 'DATETIME']):
                    entities.append(entity['span_text'])
    except Exception as e:
        continue

In [16]:
print(len(entities))

2990


#### Evaluating leakage of entities

In [7]:
from sdgeval.privacy.privacy_metrics.metrics import entity_leakage, search_phrase

In [ ]:
total_leakage, privacy_analysis = entity_leakage(synth_df['output_text'].tolist(), entities, 'privacy-leakage.pkl')

In [41]:
print(f"Percentage of leaked entities: {100*total_leakage:.3f} %")

Percentage of leaked entities: 1.934 %


#### Evaluating span memorization

In [18]:
t_df = pd.DataFrame({'text': synth_df['output_text'].tolist()[:10]})

phrases = search_phrase(df = t_df, patterns = entities, max_window_len = 3, text_field = 'text', save_file_path = './data/privacy/entity_context_synthetic_tab.csv')

Length: 10
Total number of entities 2990


In [21]:
total_leakage, _ = entity_leakage(real_texts['text'][:10], entities, entity_leakage_result_path = None)

{'PROCEDURE\n\nThe case originated in an application (no. 36244/06) against the Kingdom of Denmark lodged with the Court under Article 34 of the Convention for the Protection of Human Rights and Fundamental Freedoms (“the Convention”) by a Danish national, Mr Henrik Hasslund (“the applicant”), on 31 August 2006.\n\nThe applicant was represented by Mr Tyge Trier, a lawyer practising in Copenhagen. The Danish Government (“the Government”) were represented by their Agent, Ms Nina Holst-Christensen of the Ministry of Justice.\n\nOn 5 September 2007 the Acting President of the Fifth Section decided to give notice of the application to the Government. It was also decided to rule on the admissibility and merits of the application at the same time (Article 29 § 3).\n\nTHE FACTS\n\nTHE CIRCUMSTANCES OF THE CASE\n\nThe applicant was born in 1973 and lives in Les Salles Sur Verdon, France.\n\nAt the beginning of the 1990s a new concept called “tax asset stripping cases” (selskabstømmersager) came

In [22]:
print(f"Percentage of leaked entity contexts: {100*total_leakage:.3f} %")

Percentage of leaked entity contexts: 9.306 %


### Qualitative Evaluation Against Real Data

In [3]:
import pandas as pd
from sdgeval.qual.metrics import QualEval
from sdgeval.qual.arguments import MauveArgs, LMArgs, FrechetArgs
from dataclasses import dataclass
from datasets import load_from_disk


df = pd.DataFrame({})
synthetic_samples = pd.read_csv(synthetic_data_path)
real_samples = load_from_disk('./data/generate/data/tab')
real_samples = real_samples['train'].select(range(len(synthetic_samples)))

df['source'] = synthetic_samples['output_text']
df['reference'] = real_samples['text']

@dataclass
class args_temp:
    FrechetArgs:FrechetArgs
    MauveArgs:MauveArgs
    LMArgs:LMArgs

args_ = args_temp(FrechetArgs, MauveArgs, LMArgs)
qual_ob = QualEval(args_)

#### Reference-based evaluation with real data

In [4]:
qual_ob.calculate_fid_score(df)

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

FID Score:  {'real_scores': [0.8103712201118469, 0.7192772626876831, 0.7764612436294556, 0.7324837446212769, 0.8441793918609619, 0.819786787033081, 0.8289779424667358, 0.7650905251502991, 0.8358380794525146, 0.8501312732696533, 0.8120200634002686, 0.8108090758323669, 0.8291745781898499, 0.8329542875289917, 0.8243907690048218, 0.795727550983429, 0.8215754628181458, 0.7574876546859741, 0.8055042028427124, 0.8073018193244934, 0.8098828792572021, 0.8507733345031738, 0.8354434967041016, 0.767817497253418, 0.792038083076477, 0.8343132138252258, 0.7730989456176758, 0.8173545598983765, 0.8113252520561218, 0.8200899362564087, 0.7702681422233582, 0.7558777928352356, 0.8483886122703552, 0.8193485140800476, 0.8128591775894165, 0.8059676289558411, 0.8566105961799622, 0.8350997567176819, 0.847782552242279, 0.831849992275238, 0.8491925001144409, 0.7966157793998718, 0.7960700392723083, 0.7899898886680603, 0.8299795985221863, 0.7905206084251404, 0.8196612000465393, 0.8054779171943665, 0.782806932926178

In [5]:
qual_ob.calculate_mauve_score(df)

Featurizing p:   0%|          | 0/635 [00:00<?, ?it/s]

Featurizing q:   0%|          | 0/635 [00:00<?, ?it/s]

WARNING clustering 1270 points to 64 centroids: please provide at least 2496 training points


namespace(p_hist=array([0.00629921, 0.01102362, 0.01102362, 0.01259843, 0.0015748 ,
       0.02992126, 0.02992126, 0.01417323, 0.02047244, 0.05511811,
       0.00787402, 0.01417323, 0.02834646, 0.00787402, 0.02677165,
       0.01732283, 0.02204724, 0.00314961, 0.00944882, 0.00314961,
       0.01259843, 0.03149606, 0.00629921, 0.02047244, 0.01417323,
       0.00787402, 0.02677165, 0.0015748 , 0.01102362, 0.01574803,
       0.01102362, 0.00314961, 0.02677165, 0.00944882, 0.00787402,
       0.01102362, 0.03622047, 0.01732283, 0.01574803, 0.01259843,
       0.01259843, 0.02519685, 0.        , 0.        , 0.03937008,
       0.00472441, 0.02992126, 0.01732283, 0.00472441, 0.0015748 ,
       0.02362205, 0.01102362, 0.02519685, 0.        , 0.0503937 ,
       0.00314961, 0.00314961, 0.01259843, 0.00787402, 0.0015748 ,
       0.04566929, 0.00629921, 0.02362205, 0.01889764]), q_hist=array([0.04724409, 0.00787402, 0.00944882, 0.00787402, 0.0488189 ,
       0.02204724, 0.00787402, 0.00629921, 0.015

#### Reference-free evaluation

In [9]:
qual_ob.calculate_perplexity(df)

Using the latest cached version of the module from /home/kramesh3/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--perplexity/8ab643ad86f568b7d1d5f7822373fa7401ff5ff0297ccf114b0ca6a33be96bc0 (last modified on Thu Dec 19 16:22:06 2024) since it couldn't be found locally at evaluate-metric--perplexity, or remotely on the Hugging Face Hub.


  0%|          | 0/40 [00:00<?, ?it/s]

In [14]:
print(f"Average perplexity: {qual_ob.results['perplexity']['mean_perplexity']:.2f}")

Average perplexity: 16.77


### Miscellaneous
